In [ ]:
from google.cloud import bigquery

# Initialize the BigQuery client
client = bigquery.Client()

# Define the SQL query
query = """
    SELECT *
    FROM `geoalgo-208508.roopya_analytics_dw.CF_Mixed`
"""
# Run the query
query_job = client.query(query)
results = query_job.result()

# Convert the query results into a DataFrame
df = results.to_dataframe()

In [ ]:
df.columns

Index(['CREDIT_REPORT_ID', 'INCOME_BAND', 'PRIMARY_NO_OF_ACCOUNTS',
       'PRIMARY_ACTIVE_ACCOUNTS', 'PRIMARY_OVERDUE_ACCOUNTS',
       'PRIMARY_CURRENT_BALANCE', 'PRIMARY_SANCTIONED_AMOUNT',
       'PRIMARY_DISTRIBUTED_AMOUNT', 'PRIMARY_INSTALLMENT_AMOUNT',
       'NEW_ACCOUNTS_IN_LAST_SIX_MONTHS', 'AVERAGE_ACCOUNT_AGE',
       'NO_OF_INQUIRIES', 'SELF_INDICATOR', 'MATCH_TYPE', 'ACCOUNT_TYPE',
       'CONTRIBUTOR_TYPE', 'OWNERSHIP_IND', 'DATE_REPORTED', 'ACCOUNT_STATUS',
       'DISBURSED_DATE', 'CURRENT_BALANCE', 'OVERDUE_AMOUNT',
       'DAYS_PAST_DUE_HISTORY_MONTH_1', 'DAYS_PAST_DUE_HISTORY_MONTH_2',
       'DAYS_PAST_DUE_HISTORY_MONTH_3', 'DAYS_PAST_DUE_HISTORY_MONTH_4',
       'DAYS_PAST_DUE_HISTORY_MONTH_5', 'DAYS_PAST_DUE_HISTORY_MONTH_6',
       'DAYS_PAST_DUE_HISTORY_MONTH_7', 'DAYS_PAST_DUE_HISTORY_MONTH_8',
       'DAYS_PAST_DUE_HISTORY_MONTH_9', 'DAYS_PAST_DUE_HISTORY_MONTH_10',
       'DAYS_PAST_DUE_HISTORY_MONTH_11', 'DAYS_PAST_DUE_HISTORY_MONTH_12',
       'DAYS_PAST_D

In [ ]:
# Replace XXX and DDD values from the dataframe
def process_dpd_column(column):
    processed_values = []
    for value in column:
        value = int(value) if value.isdigit() else 0
        value = max(value - 30, 0)
        processed_values.append(value)
    return processed_values

for col in df.columns:
    if col.startswith('DAYS_PAST_DUE_HISTORY_MONTH_'):
        df[col] = process_dpd_column(df[col])

In [ ]:
dpd_col = [f'DAYS_PAST_DUE_HISTORY_MONTH_{i}' for i in range(1, 37)]
dpd_df = df[dpd_col].copy()

amt_col = [f'AMOUNT_OVERDUE_HISTORY_MONTH_{i}' for i in range(1, 37)]
amt_df = df[amt_col].copy()

import pandas as pd
m_df = pd.concat([dpd_df, amt_df], axis = 1)

In [ ]:
dpd_df = None
amt_df = None

In [ ]:
# Initialize a column for customer categorization
m_df['ROOPYA_ACCOUNT_STATUS'] = 'Uncategorised'

# Condition 1: Last 36 months all DPD is 0
condition_1 = m_df.iloc[:, 1:37].isin([0]).all(axis=1)  

m_df.loc[condition_1, 'ROOPYA_ACCOUNT_STATUS'] = 'Good'

# Condition 2: Last 6 months DPD 0 and rest DPD amount < 5000
condition_2 = ((m_df.iloc[:, 1:7].isin([0]).all(axis=1)) &
                (m_df.iloc[:, 43:73].applymap(lambda x: float(x) if str(x).isdigit() else 0).sum(axis=1) < 5000))

m_df.loc[condition_2, 'ROOPYA_ACCOUNT_STATUS'] = 'Good'

# Condition 3: Last 6 months any DPD is more than 0
condition_3 = (~m_df.iloc[:, 1:7].isin([0]).all(axis=1))

m_df.loc[condition_3, 'ROOPYA_ACCOUNT_STATUS'] = 'Bad'

# Condition 4: Last 36 months any DPD > 5000
condition_4 = (m_df.iloc[:, 37:73].applymap(lambda x: float(x) if str(x).isdigit() else 0) > 5000).any(axis=1)

m_df.loc[condition_4, 'ROOPYA_ACCOUNT_STATUS'] = 'Bad'

In [ ]:
cs_col = ['ROOPYA_ACCOUNT_STATUS']
cs_df = m_df[cs_col].copy()

In [ ]:
m_df = None 

In [ ]:
final_df = pd.concat([df, cs_df], axis = 1)
#df = None
cs_df = None

In [ ]:
# List of columns to remove
columns_to_remove = ['SELF_INDICATOR', 'MATCH_TYPE', 'DISBURSED_DATE', 'DATE_REPORTED', 'DAYS_PAST_DUE_HISTORY_MONTH_1', 'DAYS_PAST_DUE_HISTORY_MONTH_2','DAYS_PAST_DUE_HISTORY_MONTH_3', 'DAYS_PAST_DUE_HISTORY_MONTH_4','DAYS_PAST_DUE_HISTORY_MONTH_5', 'DAYS_PAST_DUE_HISTORY_MONTH_6','DAYS_PAST_DUE_HISTORY_MONTH_7', 'DAYS_PAST_DUE_HISTORY_MONTH_8','DAYS_PAST_DUE_HISTORY_MONTH_9', 'DAYS_PAST_DUE_HISTORY_MONTH_10','DAYS_PAST_DUE_HISTORY_MONTH_11', 'DAYS_PAST_DUE_HISTORY_MONTH_12','DAYS_PAST_DUE_HISTORY_MONTH_13', 'DAYS_PAST_DUE_HISTORY_MONTH_14','DAYS_PAST_DUE_HISTORY_MONTH_15', 'DAYS_PAST_DUE_HISTORY_MONTH_16','DAYS_PAST_DUE_HISTORY_MONTH_17', 'DAYS_PAST_DUE_HISTORY_MONTH_18','DAYS_PAST_DUE_HISTORY_MONTH_19', 'DAYS_PAST_DUE_HISTORY_MONTH_20', 'DAYS_PAST_DUE_HISTORY_MONTH_21', 'DAYS_PAST_DUE_HISTORY_MONTH_22','DAYS_PAST_DUE_HISTORY_MONTH_23', 'DAYS_PAST_DUE_HISTORY_MONTH_24', 'DAYS_PAST_DUE_HISTORY_MONTH_25', 'DAYS_PAST_DUE_HISTORY_MONTH_26','DAYS_PAST_DUE_HISTORY_MONTH_27', 'DAYS_PAST_DUE_HISTORY_MONTH_28','DAYS_PAST_DUE_HISTORY_MONTH_29', 'DAYS_PAST_DUE_HISTORY_MONTH_30','DAYS_PAST_DUE_HISTORY_MONTH_31', 'DAYS_PAST_DUE_HISTORY_MONTH_32','DAYS_PAST_DUE_HISTORY_MONTH_33', 'DAYS_PAST_DUE_HISTORY_MONTH_34','DAYS_PAST_DUE_HISTORY_MONTH_35', 'DAYS_PAST_DUE_HISTORY_MONTH_36','AMOUNT_OVERDUE_HISTORY_MONTH_1', 'AMOUNT_OVERDUE_HISTORY_MONTH_2','AMOUNT_OVERDUE_HISTORY_MONTH_3', 'AMOUNT_OVERDUE_HISTORY_MONTH_4','AMOUNT_OVERDUE_HISTORY_MONTH_5', 'AMOUNT_OVERDUE_HISTORY_MONTH_6','AMOUNT_OVERDUE_HISTORY_MONTH_7', 'AMOUNT_OVERDUE_HISTORY_MONTH_8','AMOUNT_OVERDUE_HISTORY_MONTH_9', 'AMOUNT_OVERDUE_HISTORY_MONTH_10','AMOUNT_OVERDUE_HISTORY_MONTH_11', 'AMOUNT_OVERDUE_HISTORY_MONTH_12','AMOUNT_OVERDUE_HISTORY_MONTH_13', 'AMOUNT_OVERDUE_HISTORY_MONTH_14','AMOUNT_OVERDUE_HISTORY_MONTH_15', 'AMOUNT_OVERDUE_HISTORY_MONTH_16','AMOUNT_OVERDUE_HISTORY_MONTH_17', 'AMOUNT_OVERDUE_HISTORY_MONTH_18','AMOUNT_OVERDUE_HISTORY_MONTH_19', 'AMOUNT_OVERDUE_HISTORY_MONTH_20','AMOUNT_OVERDUE_HISTORY_MONTH_21', 'AMOUNT_OVERDUE_HISTORY_MONTH_22','AMOUNT_OVERDUE_HISTORY_MONTH_23', 'AMOUNT_OVERDUE_HISTORY_MONTH_24','AMOUNT_OVERDUE_HISTORY_MONTH_25', 'AMOUNT_OVERDUE_HISTORY_MONTH_26','AMOUNT_OVERDUE_HISTORY_MONTH_27', 'AMOUNT_OVERDUE_HISTORY_MONTH_28','AMOUNT_OVERDUE_HISTORY_MONTH_29', 'AMOUNT_OVERDUE_HISTORY_MONTH_30','AMOUNT_OVERDUE_HISTORY_MONTH_31', 'AMOUNT_OVERDUE_HISTORY_MONTH_32','AMOUNT_OVERDUE_HISTORY_MONTH_33', 'AMOUNT_OVERDUE_HISTORY_MONTH_34','AMOUNT_OVERDUE_HISTORY_MONTH_35', 'AMOUNT_OVERDUE_HISTORY_MONTH_36', 'AMOUNT']

# Remove the specified columns
final_df_cleaned = final_df.drop(columns=columns_to_remove)

In [ ]:
file_path_w = 'gs://roopya_analytics_workarea/Consolidate_Customer_Preprocessing_1.csv'
final_df_cleaned.to_csv(file_path_w)
print("DONE")

DONE


In [ ]:
from google.cloud import bigquery

# Initialize the BigQuery client
client = bigquery.Client()

# Define the SQL query
query = """
    SELECT *
    FROM `geoalgo-208508.roopya_analytics_dw.Consolidate_Customer_Preprocessing_1`
"""
# Run the query
query_job = client.query(query)
results = query_job.result()

# Convert the query results into a DataFrame
final_df = results.to_dataframe()

NotFound: 404 Not found: Table geoalgo-208508:roopya_analytics_dw.Consolidate_Customer_Preprocessing_1 was not found in location asia-south1

Location: asia-south1
Job ID: 0b67415e-94f9-4d47-9f52-cea90a5e8dc4


In [ ]:
final_df = final_df_cleaned

In [ ]:
final_df.shape

(10547762, 22)

In [ ]:
final_df["CURRENT_BALANCE"].isnull().sum()

3920

In [ ]:
# List of columns to replace
columns_to_process = ['CURRENT_BALANCE', 'INCOME', 'OVERDUE_AMOUNT']

# Replace null by 0 
final_df[columns_to_process] = final_df[columns_to_process].fillna(0)

In [ ]:
final_df1 = final_df[(final_df['CURRENT_BALANCE'] >= 0) & (final_df['CURRENT_BALANCE'] <= 10000000) & (final_df['PRIMARY_NO_OF_ACCOUNTS'] <= 100)]

In [ ]:
final_df1.shape

(10084589, 22)

In [ ]:
df = final_df1

In [ ]:
#df.shape

In [ ]:
#df = df.drop('int64_field_0', axis = 1)

In [ ]:
import pandas as pd
pd.set_option('display.float_format', '{:.2f}'.format)
df.describe()

,PRIMARY_NO_OF_ACCOUNTS,PRIMARY_ACTIVE_ACCOUNTS,PRIMARY_OVERDUE_ACCOUNTS,PRIMARY_CURRENT_BALANCE,PRIMARY_SANCTIONED_AMOUNT,PRIMARY_DISTRIBUTED_AMOUNT,PRIMARY_INSTALLMENT_AMOUNT,NEW_ACCOUNTS_IN_LAST_SIX_MONTHS,NO_OF_INQUIRIES,CURRENT_BALANCE,OVERDUE_AMOUNT,INCOME,AGE_COHORT
count,10084589.00,10084589.00,10084589.00,10084589.00,10084589.00,10084589.00,10084589.00,10084589.00,10084589.00,10084589.00,10084589.00,10084589.00,10070564.00
mean,30.74,10.50,1.29,7568685.29,12145706.19,11962516.51,350836.25,3.51,1.63,166859.46,27.34,2302322.25,40.93
std,23.17,7.82,2.95,67439175.03,752916951.07,752923922.75,12790069.91,6.47,3.02,702598.18,4575.00,35793314.84,6.79
min,1.00,0.00,0.00,-559899904.00,0.00,0.00,-331462.00,0.00,0.00,0.00,-48300.00,0.00,20.00
25%,13.00,5.00,0.00,302351.00,763773.00,643390.00,13476.00,0.00,0.00,0.00,0.00,0.00,35.00
50%,24.00,9.00,0.00,1486607.00,2554540.00,2367025.00,44047.00,1.00,0.00,0.00,0.00,0.00,40.00
75%,43.00,14.00,1.00,4300743.00,6381482.00,6061585.00,128464.00,4.00,2.00,21850.00,0.00,0.00,45.00
max,99.00,82.00,78.00,26802096128.00,748806995968.00,748806995968.00,10844999649.00,65.00,42.00,10000000.00,5533702.00,999999000.00,60.00


In [ ]:
df1 = df[(df['PRIMARY_CURRENT_BALANCE'] >= 0) & (df['PRIMARY_CURRENT_BALANCE'] <= 10000000) & (df['PRIMARY_SANCTIONED_AMOUNT'] >= 0) & (df['PRIMARY_SANCTIONED_AMOUNT'] <= 10000000) & (df['PRIMARY_DISTRIBUTED_AMOUNT'] >= 0) & (df['PRIMARY_DISTRIBUTED_AMOUNT'] <= 10000000) & (df['PRIMARY_INSTALLMENT_AMOUNT'] >= 0) & (df['PRIMARY_INSTALLMENT_AMOUNT'] <= 10000000) & (df['INCOME'] >= 0) & (df['INCOME'] <= 10000000)]

In [ ]:
pd.set_option('display.float_format', '{:.2f}'.format)
df1.describe()

,PRIMARY_NO_OF_ACCOUNTS,PRIMARY_ACTIVE_ACCOUNTS,PRIMARY_OVERDUE_ACCOUNTS,PRIMARY_CURRENT_BALANCE,PRIMARY_SANCTIONED_AMOUNT,PRIMARY_DISTRIBUTED_AMOUNT,PRIMARY_INSTALLMENT_AMOUNT,NEW_ACCOUNTS_IN_LAST_SIX_MONTHS,NO_OF_INQUIRIES,CURRENT_BALANCE,OVERDUE_AMOUNT,INCOME,AGE_COHORT
count,8412439.00,8412439.00,8412439.00,8412439.00,8412439.00,8412439.00,8412439.00,8412439.00,8412439.00,8412439.00,8412439.00,8412439.00,8405166.00
mean,29.71,9.64,1.36,1784076.41,2698971.87,2533410.88,132717.52,3.72,1.63,92397.99,19.50,13720.80,40.46
std,23.31,7.13,3.07,1967066.32,2540101.81,2459734.52,422719.13,6.89,3.10,412419.70,3635.11,153390.89,6.68
min,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-11070.00,0.00,20.00
25%,12.00,4.00,0.00,217136.00,597936.00,502184.00,11018.00,0.00,0.00,0.00,0.00,0.00,35.00
50%,22.00,8.00,0.00,1044624.00,1876221.00,1706338.00,34884.00,1.00,0.00,0.00,0.00,0.00,40.00
75%,41.00,13.00,1.00,2753324.00,4186708.00,3957500.00,88216.00,4.00,2.00,14107.00,0.00,0.00,45.00
max,99.00,56.00,78.00,10000000.00,10000000.00,10000000.00,9992561.00,65.00,42.00,10000000.00,2740526.00,10000000.00,60.00


In [ ]:
df1.columns

Index(['CREDIT_REPORT_ID', 'INCOME_BAND', 'PRIMARY_NO_OF_ACCOUNTS',
       'PRIMARY_ACTIVE_ACCOUNTS', 'PRIMARY_OVERDUE_ACCOUNTS',
       'PRIMARY_CURRENT_BALANCE', 'PRIMARY_SANCTIONED_AMOUNT',
       'PRIMARY_DISTRIBUTED_AMOUNT', 'PRIMARY_INSTALLMENT_AMOUNT',
       'NEW_ACCOUNTS_IN_LAST_SIX_MONTHS', 'AVERAGE_ACCOUNT_AGE',
       'NO_OF_INQUIRIES', 'ACCOUNT_TYPE', 'CONTRIBUTOR_TYPE', 'OWNERSHIP_IND',
       'ACCOUNT_STATUS', 'CURRENT_BALANCE', 'OVERDUE_AMOUNT', 'INCOME',
       'AGE_COHORT', 'STATE', 'ROOPYA_ACCOUNT_STATUS'],
      dtype='object')

In [ ]:
df = df1.copy()

In [ ]:
# Specify the columns to consider for duplicates
columns_to_check = ['CREDIT_REPORT_ID','PRIMARY_NO_OF_ACCOUNTS', 'PRIMARY_ACTIVE_ACCOUNTS', 'PRIMARY_OVERDUE_ACCOUNTS', 'PRIMARY_CURRENT_BALANCE', 'PRIMARY_SANCTIONED_AMOUNT', 'PRIMARY_DISTRIBUTED_AMOUNT', 'PRIMARY_INSTALLMENT_AMOUNT', 'NEW_ACCOUNTS_IN_LAST_SIX_MONTHS', 'AVERAGE_ACCOUNT_AGE', 'NO_OF_INQUIRIES', 'INCOME', 'AGE_COHORT', 'STATE']

# Drop duplicates based on the specified columns
cl_df = df.drop_duplicates(subset='CREDIT_REPORT_ID')

In [ ]:
df.columns[df.isnull().any()]

Index(['INCOME_BAND', 'AGE_COHORT'], dtype='object')

In [ ]:
# Define loan categories
loan_categories = {
    'Credit Card': 'Credit Card', 'Loan on Credit Card': 'Credit Card', 'Secured Credit Card': 'Credit Card',
    'Corporate Credit Card': 'Credit Card', 'Kisan Credit Card': 'Credit Card', 'Overdraft': 'Personal Loan',
    'Personal Loan': 'Personal Loan', 'Consumer Loan': 'Personal Loan', 
    'Loan Aganist Bank Deposits': 'Personal Loan', 'OD on Savings Account': 'Personal Loan', 
    'Microfinance Personal Loan': 'Personal Loan', 'Loan to Professional': 'Personal Loan',
    'Auto Loan (Personal)': 'Auto Loan', 'Two Wheeler Loan': 'Auto Loan', 'Used Car Loan': 'Auto Loan',
    'Commercial Vehicle Loan': 'Auto Loan', 'Used Tractor Loan': 'Auto Loan', 
    'Housing Loan': 'Home Loan', 'Property Loan': 'Home Loan', 'Leasing': 'Home Loan',
    'Microfinance Housing Loan': 'Home Loan'
}

# Map the loan categories to the 'ACCOUNT_TYPE' column
df['Loan Category'] = df['ACCOUNT_TYPE'].map(loan_categories)

# Pivot the DataFrame to create separate columns for each loan category
acc_df = df.pivot_table(index='CREDIT_REPORT_ID', columns='Loan Category', aggfunc= 'size', fill_value=0)

# Reset the index to make 'CREDIT_REPORT_ID' a column again
acc_df.reset_index(inplace=True)

# Rename columns for better readability
acc_df.rename(columns={'Credit Card': 'ACCOUNT_TYPE_Credit Card', 'Home loan': 'ACCOUNT_TYPE_Home Loan', 'Auto Loan': 'ACCOUNT_TYPE_Auto Loan', 'Personal Loan': 'ACCOUNT_TYPE_Personal Loan'}, inplace=True)

In [ ]:
merged_df = acc_df.merge(df, on="CREDIT_REPORT_ID", how="left")

In [ ]:
merged_df.shape

(8326868, 27)

In [ ]:
merged_df["CREDIT_REPORT_ID"].duplicated().sum()

7222626

In [ ]:
acc_df.shape

(1104242, 5)

In [ ]:
merged_df.columns[merged_df.isnull().any()]

Index(['INCOME_BAND', 'AGE_COHORT', 'Loan Category'], dtype='object')

In [ ]:
acty_df = merged_df.drop_duplicates(subset='CREDIT_REPORT_ID')

In [ ]:
acty_df.shape

(1104242, 27)

In [ ]:
# Pivot CONTRIBUTOR_TYPE data
ct_df = df.pivot_table(index = 'CREDIT_REPORT_ID', columns = 'CONTRIBUTOR_TYPE', aggfunc ='size', fill_value = 0)

# Reset index to have 'CREDIT_REPORT_ID' as a regular column
ct_df.reset_index(inplace=True)

# Rename columns for better readability
ct_df.rename(columns={'PRB': 'CONTRIBUTOR_TYPE_PRB', 'NBF': 'CONTRIBUTOR_TYPE_NBF'}, inplace=True)

In [ ]:
merged_df = acty_df.merge(ct_df, on="CREDIT_REPORT_ID", how="left")

In [ ]:
# Pivot OWNERSHIP_IND data
oi_df = df.pivot_table(index = 'CREDIT_REPORT_ID', columns = 'OWNERSHIP_IND', aggfunc = 'size', fill_value = 0)

# Reset index to have 'CREDIT_REPORT_ID' as a regular column
oi_df.reset_index(inplace=True)

# Rename columns for better readability
oi_df.rename(columns={'Individual': 'OWNERSHIP_IND_Individual', 'Supl Card Holder': 'OWNERSHIP_IND_Supl Card Holder', 'Joint': 'OWNERSHIP_IND_Joint', 'Guarantor': 'OWNERSHIP_IND_Guarantor'}, inplace=True)

In [ ]:
merged1_df = merged_df.merge(oi_df, on="CREDIT_REPORT_ID", how="left")

In [ ]:
# Pivot ACCOUNT_STATUS data
as_df = df.pivot_table(index='CREDIT_REPORT_ID', columns='ACCOUNT_STATUS', aggfunc='size', fill_value=0)

# Reset index and remove the name of the columns index
as_df.reset_index(inplace=True)

# Rename columns for better readability
as_df.rename(columns={'Active': 'ACCOUNT_STATUS_Active', 'Closed': 'ACCOUNT_STATUS_Closed'}, inplace=True)

In [ ]:
merged2_df = merged1_df.merge(as_df, on="CREDIT_REPORT_ID", how="left")

In [ ]:
merged2_df.head(5)

,CREDIT_REPORT_ID,ACCOUNT_TYPE_Auto Loan,ACCOUNT_TYPE_Credit Card,Home Loan,ACCOUNT_TYPE_Personal Loan,INCOME_BAND,PRIMARY_NO_OF_ACCOUNTS,PRIMARY_ACTIVE_ACCOUNTS,PRIMARY_OVERDUE_ACCOUNTS,PRIMARY_CURRENT_BALANCE,...,Loan Category,CONTRIBUTOR_TYPE_NBF,CONTRIBUTOR_TYPE_PRB,OWNERSHIP_IND_Guarantor,OWNERSHIP_IND_Individual,OWNERSHIP_IND_Joint,Primary,OWNERSHIP_IND_Supl Card Holder,ACCOUNT_STATUS_Active,ACCOUNT_STATUS_Closed
0,GEOA221114CR000011523894971,0,1,0,0,H,4.00,2.00,0.00,12737.00,...,Credit Card,0,1,0,1,0,0,0,1,0
1,GEOA221114CR000111523894971,0,1,0,0,D,1.00,1.00,0.00,235.00,...,Credit Card,0,1,0,1,0,0,0,1,0
2,GEOA221114CR000211523894971,0,1,0,0,H,9.00,5.00,0.00,3052319.00,...,Credit Card,0,1,0,1,0,0,0,1,0
3,GEOA221114CR00021523894971,0,5,0,14,H,46.00,14.00,0.00,835826.00,...,NaN,15,6,0,21,0,0,0,9,12
4,GEOA221114CR00041523894971,0,0,0,3,F,15.00,5.00,4.00,399263.00,...,Personal Loan,5,0,0,5,0,0,0,1,4


In [ ]:
agg_data = df[['CREDIT_REPORT_ID', 'ACCOUNT_STATUS', 'CURRENT_BALANCE', 'OVERDUE_AMOUNT']]

def aggregation(df):
    # Create a pivot table to aggregate balances and overdue amounts
    pivot_df = df.pivot_table(index='CREDIT_REPORT_ID', columns='ACCOUNT_STATUS', 
                              values=['CURRENT_BALANCE', 'OVERDUE_AMOUNT'], 
                              aggfunc={'CURRENT_BALANCE': 'sum', 'OVERDUE_AMOUNT': 'sum'}, 
                              fill_value=0)
    
    # Flatten the multi-level column index
    pivot_df.columns = [f'{agg}_{status}' for status, agg in pivot_df.columns]
    
    # Reset the index to make 'CREDIT_REPORT_ID' a regular column
    pivot_df.reset_index(inplace=True)
    
    return pivot_df

# Call the function to aggregate balances
aggregated_data = aggregation(agg_data)

In [ ]:
merged3_df = merged2_df.merge(aggregated_data, on="CREDIT_REPORT_ID", how="left")

In [ ]:
merged2_df = merged2_df.drop(['CURRENT_BALANCE', 'OVERDUE_AMOUNT'], axis = 1)

In [ ]:
merged3_df["ROOPYA_ACCOUNT_STATUS"].value_counts()

Good    1084640
Bad       19602
Name: ROOPYA_ACCOUNT_STATUS, dtype: int64

In [ ]:
cri_df = merged3_df[['CREDIT_REPORT_ID', 'ROOPYA_ACCOUNT_STATUS']]

# Custom function to determine the final status
def categorize_customer(group):
    unique_statuses = group['ROOPYA_ACCOUNT_STATUS'].unique()

    if len(unique_statuses) == 1:
        return unique_statuses[0]
    elif 'Bad' in unique_statuses:
        return 'Bad'
    else:
        return 'Good'

result = cri_df.groupby('CREDIT_REPORT_ID').apply(categorize_customer)

# Create a new DataFrame with the summarized results
final_status_df = result.reset_index(name='ROOPYA_CUSTOMER_STATUS')

In [ ]:
merged3_df.columns

Index(['CREDIT_REPORT_ID', 'ACCOUNT_TYPE_Auto Loan',
       'ACCOUNT_TYPE_Credit Card', 'Home Loan', 'ACCOUNT_TYPE_Personal Loan',
       'INCOME_BAND', 'PRIMARY_NO_OF_ACCOUNTS', 'PRIMARY_ACTIVE_ACCOUNTS',
       'PRIMARY_OVERDUE_ACCOUNTS', 'PRIMARY_CURRENT_BALANCE',
       'PRIMARY_SANCTIONED_AMOUNT', 'PRIMARY_DISTRIBUTED_AMOUNT',
       'PRIMARY_INSTALLMENT_AMOUNT', 'NEW_ACCOUNTS_IN_LAST_SIX_MONTHS',
       'AVERAGE_ACCOUNT_AGE', 'NO_OF_INQUIRIES', 'ACCOUNT_TYPE',
       'CONTRIBUTOR_TYPE', 'OWNERSHIP_IND', 'ACCOUNT_STATUS',
       'CURRENT_BALANCE', 'OVERDUE_AMOUNT', 'INCOME', 'AGE_COHORT', 'STATE',
       'ROOPYA_ACCOUNT_STATUS', 'Loan Category', 'CONTRIBUTOR_TYPE_NBF',
       'CONTRIBUTOR_TYPE_PRB', 'OWNERSHIP_IND_Guarantor',
       'OWNERSHIP_IND_Individual', 'OWNERSHIP_IND_Joint', 'Primary',
       'OWNERSHIP_IND_Supl Card Holder', 'ACCOUNT_STATUS_Active',
       'ACCOUNT_STATUS_Closed', 'Active_CURRENT_BALANCE',
       'Closed_CURRENT_BALANCE', 'Active_OVERDUE_AMOUNT',
       '

In [ ]:
df_1 = merged3_df.drop(columns = ['INCOME_BAND', 'ACCOUNT_TYPE', 'CONTRIBUTOR_TYPE', 'OWNERSHIP_IND', 'ACCOUNT_STATUS','ROOPYA_ACCOUNT_STATUS', 'Loan Category', 'Primary'])
df_2 = final_status_df.drop(columns = ['CREDIT_REPORT_ID'])

In [ ]:
df1 = df_1.reset_index(drop=True)
df2 = df_2.reset_index(drop=True)

In [ ]:
import pandas as pd
result_df = pd.concat([df1, df2], axis = 1)

In [ ]:
final_df = result_df.rename(columns={'Home Loan': 'ACCOUNT_TYPE_Home Loan'})

In [ ]:
final_df.isnull().sum()

CREDIT_REPORT_ID                      0
ACCOUNT_TYPE_Auto Loan                0
ACCOUNT_TYPE_Credit Card              0
ACCOUNT_TYPE_Home Loan                0
ACCOUNT_TYPE_Personal Loan            0
PRIMARY_NO_OF_ACCOUNTS                0
PRIMARY_ACTIVE_ACCOUNTS               0
PRIMARY_OVERDUE_ACCOUNTS              0
PRIMARY_CURRENT_BALANCE               0
PRIMARY_SANCTIONED_AMOUNT             0
PRIMARY_DISTRIBUTED_AMOUNT            0
PRIMARY_INSTALLMENT_AMOUNT            0
NEW_ACCOUNTS_IN_LAST_SIX_MONTHS       0
AVERAGE_ACCOUNT_AGE                   0
NO_OF_INQUIRIES                       0
CURRENT_BALANCE                       0
OVERDUE_AMOUNT                        0
INCOME                                0
AGE_COHORT                         2034
STATE                                 0
CONTRIBUTOR_TYPE_NBF                  0
CONTRIBUTOR_TYPE_PRB                  0
OWNERSHIP_IND_Guarantor               0
OWNERSHIP_IND_Individual              0
OWNERSHIP_IND_Joint                   0


In [ ]:
final_df['ROOPYA_CUSTOMER_STATUS'].value_counts()

Good    1084640
Bad       19602
Name: ROOPYA_CUSTOMER_STATUS, dtype: int64

In [ ]:
print("No of customers with Auto Loan: ", len(final_df[final_df["ACCOUNT_TYPE_Auto Loan"] != 0]))
print("No of customers with Credit Card: ", len(final_df[final_df["ACCOUNT_TYPE_Credit Card"] != 0]))
print("No of customers with Home Loan: ", len(final_df[final_df["ACCOUNT_TYPE_Home Loan"] != 0]))
print("No of customers with Personal Loan: ", len(final_df[final_df["ACCOUNT_TYPE_Personal Loan"] != 0]))

No of customers with Auto Loan:  160915
No of customers with Credit Card:  831555
No of customers with Home Loan:  122333
No of customers with Personal Loan:  625293


In [ ]:
pd.set_option('display.float_format', '{:.2f}'.format)
final_df.describe()

,ACCOUNT_TYPE_Auto Loan,ACCOUNT_TYPE_Credit Card,ACCOUNT_TYPE_Home Loan,ACCOUNT_TYPE_Personal Loan,PRIMARY_NO_OF_ACCOUNTS,PRIMARY_ACTIVE_ACCOUNTS,PRIMARY_OVERDUE_ACCOUNTS,PRIMARY_CURRENT_BALANCE,PRIMARY_SANCTIONED_AMOUNT,PRIMARY_DISTRIBUTED_AMOUNT,...,OWNERSHIP_IND_Guarantor,OWNERSHIP_IND_Individual,OWNERSHIP_IND_Joint,OWNERSHIP_IND_Supl Card Holder,ACCOUNT_STATUS_Active,ACCOUNT_STATUS_Closed,Active_CURRENT_BALANCE,Closed_CURRENT_BALANCE,Active_OVERDUE_AMOUNT,Closed_OVERDUE_AMOUNT
count,1104242.00,1104242.00,1104242.00,1104242.00,1104242.00,1104242.00,1104242.00,1104242.00,1104242.00,1104242.00,...,1104242.00,1104242.00,1104242.00,1104242.00,1104242.00,1104242.00,1104242.00,1104242.00,1104242.00,1104242.00
mean,0.31,2.04,0.18,4.25,11.27,4.16,0.39,1225470.92,2100587.28,1932027.73,...,0.05,7.19,0.27,0.02,3.94,3.60,693476.74,968.39,7.35,128.34
std,1.57,4.14,0.84,27.44,10.19,3.66,1.24,1767765.01,2412553.14,2343121.10,...,0.62,31.90,1.34,0.17,11.75,22.97,2534252.76,67589.60,325.88,20394.24
min,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-44280.00,-18.00
25%,0.00,1.00,0.00,0.00,5.00,2.00,0.00,17144.25,205991.50,129833.25,...,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00
50%,0.00,1.00,0.00,1.00,9.00,3.00,0.00,332962.00,1075000.00,849025.50,...,0.00,3.00,0.00,0.00,2.00,1.00,27891.00,0.00,0.00,0.00
75%,0.00,2.00,0.00,3.00,15.00,6.00,0.00,1838420.75,3300564.25,3081971.25,...,0.00,5.00,0.00,0.00,4.00,2.00,318380.75,0.00,0.00,0.00
max,180.00,484.00,235.00,3375.00,99.00,56.00,78.00,10000000.00,10000000.00,10000000.00,...,186.00,3750.00,141.00,29.00,1364.00,2628.00,211213912.00,29681964.00,104558.00,17509303.00


In [ ]:
df10 = final_df[(final_df['Active_OVERDUE_AMOUNT'] >= 0)]

In [ ]:
df10 = df10.drop(columns = ['Closed_CURRENT_BALANCE', 'Closed_OVERDUE_AMOUNT'], axis = 1)

In [ ]:
df10.columns

Index(['CREDIT_REPORT_ID', 'ACCOUNT_TYPE_Auto Loan',
       'ACCOUNT_TYPE_Credit Card', 'ACCOUNT_TYPE_Home Loan',
       'ACCOUNT_TYPE_Personal Loan', 'PRIMARY_NO_OF_ACCOUNTS',
       'PRIMARY_ACTIVE_ACCOUNTS', 'PRIMARY_OVERDUE_ACCOUNTS',
       'PRIMARY_CURRENT_BALANCE', 'PRIMARY_SANCTIONED_AMOUNT',
       'PRIMARY_DISTRIBUTED_AMOUNT', 'PRIMARY_INSTALLMENT_AMOUNT',
       'NEW_ACCOUNTS_IN_LAST_SIX_MONTHS', 'AVERAGE_ACCOUNT_AGE',
       'NO_OF_INQUIRIES', 'CURRENT_BALANCE', 'OVERDUE_AMOUNT', 'INCOME',
       'AGE_COHORT', 'STATE', 'CONTRIBUTOR_TYPE_NBF', 'CONTRIBUTOR_TYPE_PRB',
       'OWNERSHIP_IND_Guarantor', 'OWNERSHIP_IND_Individual',
       'OWNERSHIP_IND_Joint', 'OWNERSHIP_IND_Supl Card Holder',
       'ACCOUNT_STATUS_Active', 'ACCOUNT_STATUS_Closed',
       'Active_CURRENT_BALANCE', 'Active_OVERDUE_AMOUNT',
       'ROOPYA_CUSTOMER_STATUS'],
      dtype='object')

In [ ]:
file_path_w = 'gs://roopya_analytics_workarea/Consolidated_Customer.csv'
df10.to_csv(file_path_w)
print("DONE")

DONE


In [ ]:
from google.cloud import bigquery

# Initialize the BigQuery client
client = bigquery.Client()

# Define the SQL query
query = """
    SELECT *
    FROM `geoalgo-208508.roopya_analytics_dw.CF_Mixed`
"""
# Run the query
query_job = client.query(query)
results = query_job.result()

# Convert the query results into a DataFrame
df = results.to_dataframe()

In [ ]:
df.columns

Index(['CREDIT_REPORT_ID', 'INCOME_BAND', 'PRIMARY_NO_OF_ACCOUNTS',
       'PRIMARY_ACTIVE_ACCOUNTS', 'PRIMARY_OVERDUE_ACCOUNTS',
       'PRIMARY_CURRENT_BALANCE', 'PRIMARY_SANCTIONED_AMOUNT',
       'PRIMARY_DISTRIBUTED_AMOUNT', 'PRIMARY_INSTALLMENT_AMOUNT',
       'NEW_ACCOUNTS_IN_LAST_SIX_MONTHS', 'AVERAGE_ACCOUNT_AGE',
       'NO_OF_INQUIRIES', 'SELF_INDICATOR', 'MATCH_TYPE', 'ACCOUNT_TYPE',
       'CONTRIBUTOR_TYPE', 'OWNERSHIP_IND', 'DATE_REPORTED', 'ACCOUNT_STATUS',
       'DISBURSED_DATE', 'CURRENT_BALANCE', 'OVERDUE_AMOUNT',
       'DAYS_PAST_DUE_HISTORY_MONTH_1', 'DAYS_PAST_DUE_HISTORY_MONTH_2',
       'DAYS_PAST_DUE_HISTORY_MONTH_3', 'DAYS_PAST_DUE_HISTORY_MONTH_4',
       'DAYS_PAST_DUE_HISTORY_MONTH_5', 'DAYS_PAST_DUE_HISTORY_MONTH_6',
       'DAYS_PAST_DUE_HISTORY_MONTH_7', 'DAYS_PAST_DUE_HISTORY_MONTH_8',
       'DAYS_PAST_DUE_HISTORY_MONTH_9', 'DAYS_PAST_DUE_HISTORY_MONTH_10',
       'DAYS_PAST_DUE_HISTORY_MONTH_11', 'DAYS_PAST_DUE_HISTORY_MONTH_12',
       'DAYS_PAST_D

In [ ]:
# Replace XXX and DDD values from the dataframe
def process_dpd_column(column):
    processed_values = []
    for value in column:
        value = int(value) if value.isdigit() else 0
        value = max(value - 30, 0)
        processed_values.append(value)
    return processed_values

for col in df.columns:
    if col.startswith('DAYS_PAST_DUE_HISTORY_MONTH_'):
        df[col] = process_dpd_column(df[col])

In [ ]:
dpd_col = [f'DAYS_PAST_DUE_HISTORY_MONTH_{i}' for i in range(1, 37)]
dpd_df = df[dpd_col].copy()

amt_col = [f'AMOUNT_OVERDUE_HISTORY_MONTH_{i}' for i in range(1, 37)]
amt_df = df[amt_col].copy()

import pandas as pd
m_df = pd.concat([dpd_df, amt_df], axis = 1)

In [ ]:
dpd_df = None
amt_df = None

In [ ]:
# Initialize a column for customer categorization
m_df['ROOPYA_ACCOUNT_STATUS'] = 'Uncategorised'

# Condition 1: Last 36 months all DPD is 0
condition_1 = m_df.iloc[:, 1:37].isin([0]).all(axis=1)  

m_df.loc[condition_1, 'ROOPYA_ACCOUNT_STATUS'] = 'Good'

# Condition 2: Last 6 months DPD 0 and rest DPD amount < 5000
condition_2 = ((m_df.iloc[:, 1:7].isin([0]).all(axis=1)) &
                (m_df.iloc[:, 43:73].applymap(lambda x: float(x) if str(x).isdigit() else 0).sum(axis=1) < 5000))

m_df.loc[condition_2, 'ROOPYA_ACCOUNT_STATUS'] = 'Good'

# Condition 3: Last 6 months any DPD is more than 0
condition_3 = (~m_df.iloc[:, 1:7].isin([0]).all(axis=1))

m_df.loc[condition_3, 'ROOPYA_ACCOUNT_STATUS'] = 'Bad'

# Condition 4: Last 36 months any DPD > 5000
condition_4 = (m_df.iloc[:, 37:73].applymap(lambda x: float(x) if str(x).isdigit() else 0) > 5000).any(axis=1)

m_df.loc[condition_4, 'ROOPYA_ACCOUNT_STATUS'] = 'Bad'

In [ ]:
cs_col = ['ROOPYA_ACCOUNT_STATUS']
cs_df = m_df[cs_col].copy()

In [ ]:
m_df = None 

In [ ]:
final_df = pd.concat([df, cs_df], axis = 1)
#df = None
cs_df = None

In [ ]:
# List of columns to remove
columns_to_remove = ['SELF_INDICATOR', 'MATCH_TYPE', 'DISBURSED_DATE', 'DATE_REPORTED', 'DAYS_PAST_DUE_HISTORY_MONTH_1', 'DAYS_PAST_DUE_HISTORY_MONTH_2','DAYS_PAST_DUE_HISTORY_MONTH_3', 'DAYS_PAST_DUE_HISTORY_MONTH_4','DAYS_PAST_DUE_HISTORY_MONTH_5', 'DAYS_PAST_DUE_HISTORY_MONTH_6','DAYS_PAST_DUE_HISTORY_MONTH_7', 'DAYS_PAST_DUE_HISTORY_MONTH_8','DAYS_PAST_DUE_HISTORY_MONTH_9', 'DAYS_PAST_DUE_HISTORY_MONTH_10','DAYS_PAST_DUE_HISTORY_MONTH_11', 'DAYS_PAST_DUE_HISTORY_MONTH_12','DAYS_PAST_DUE_HISTORY_MONTH_13', 'DAYS_PAST_DUE_HISTORY_MONTH_14','DAYS_PAST_DUE_HISTORY_MONTH_15', 'DAYS_PAST_DUE_HISTORY_MONTH_16','DAYS_PAST_DUE_HISTORY_MONTH_17', 'DAYS_PAST_DUE_HISTORY_MONTH_18','DAYS_PAST_DUE_HISTORY_MONTH_19', 'DAYS_PAST_DUE_HISTORY_MONTH_20', 'DAYS_PAST_DUE_HISTORY_MONTH_21', 'DAYS_PAST_DUE_HISTORY_MONTH_22','DAYS_PAST_DUE_HISTORY_MONTH_23', 'DAYS_PAST_DUE_HISTORY_MONTH_24', 'DAYS_PAST_DUE_HISTORY_MONTH_25', 'DAYS_PAST_DUE_HISTORY_MONTH_26','DAYS_PAST_DUE_HISTORY_MONTH_27', 'DAYS_PAST_DUE_HISTORY_MONTH_28','DAYS_PAST_DUE_HISTORY_MONTH_29', 'DAYS_PAST_DUE_HISTORY_MONTH_30','DAYS_PAST_DUE_HISTORY_MONTH_31', 'DAYS_PAST_DUE_HISTORY_MONTH_32','DAYS_PAST_DUE_HISTORY_MONTH_33', 'DAYS_PAST_DUE_HISTORY_MONTH_34','DAYS_PAST_DUE_HISTORY_MONTH_35', 'DAYS_PAST_DUE_HISTORY_MONTH_36','AMOUNT_OVERDUE_HISTORY_MONTH_1', 'AMOUNT_OVERDUE_HISTORY_MONTH_2','AMOUNT_OVERDUE_HISTORY_MONTH_3', 'AMOUNT_OVERDUE_HISTORY_MONTH_4','AMOUNT_OVERDUE_HISTORY_MONTH_5', 'AMOUNT_OVERDUE_HISTORY_MONTH_6','AMOUNT_OVERDUE_HISTORY_MONTH_7', 'AMOUNT_OVERDUE_HISTORY_MONTH_8','AMOUNT_OVERDUE_HISTORY_MONTH_9', 'AMOUNT_OVERDUE_HISTORY_MONTH_10','AMOUNT_OVERDUE_HISTORY_MONTH_11', 'AMOUNT_OVERDUE_HISTORY_MONTH_12','AMOUNT_OVERDUE_HISTORY_MONTH_13', 'AMOUNT_OVERDUE_HISTORY_MONTH_14','AMOUNT_OVERDUE_HISTORY_MONTH_15', 'AMOUNT_OVERDUE_HISTORY_MONTH_16','AMOUNT_OVERDUE_HISTORY_MONTH_17', 'AMOUNT_OVERDUE_HISTORY_MONTH_18','AMOUNT_OVERDUE_HISTORY_MONTH_19', 'AMOUNT_OVERDUE_HISTORY_MONTH_20','AMOUNT_OVERDUE_HISTORY_MONTH_21', 'AMOUNT_OVERDUE_HISTORY_MONTH_22','AMOUNT_OVERDUE_HISTORY_MONTH_23', 'AMOUNT_OVERDUE_HISTORY_MONTH_24','AMOUNT_OVERDUE_HISTORY_MONTH_25', 'AMOUNT_OVERDUE_HISTORY_MONTH_26','AMOUNT_OVERDUE_HISTORY_MONTH_27', 'AMOUNT_OVERDUE_HISTORY_MONTH_28','AMOUNT_OVERDUE_HISTORY_MONTH_29', 'AMOUNT_OVERDUE_HISTORY_MONTH_30','AMOUNT_OVERDUE_HISTORY_MONTH_31', 'AMOUNT_OVERDUE_HISTORY_MONTH_32','AMOUNT_OVERDUE_HISTORY_MONTH_33', 'AMOUNT_OVERDUE_HISTORY_MONTH_34','AMOUNT_OVERDUE_HISTORY_MONTH_35', 'AMOUNT_OVERDUE_HISTORY_MONTH_36', 'AMOUNT']

# Remove the specified columns
final_df_cleaned = final_df.drop(columns=columns_to_remove)

In [ ]:
file_path_w = 'gs://roopya_analytics_workarea/Consolidate_Customer_Preprocessing_1.csv'
final_df_cleaned.to_csv(file_path_w)
print("DONE")

DONE


In [ ]:
from google.cloud import bigquery

# Initialize the BigQuery client
client = bigquery.Client()

# Define the SQL query
query = """
    SELECT *
    FROM `geoalgo-208508.roopya_analytics_dw.Consolidate_Customer_Preprocessing_1`
"""
# Run the query
query_job = client.query(query)
results = query_job.result()

# Convert the query results into a DataFrame
final_df = results.to_dataframe()

NotFound: 404 Not found: Table geoalgo-208508:roopya_analytics_dw.Consolidate_Customer_Preprocessing_1 was not found in location asia-south1

Location: asia-south1
Job ID: 0b67415e-94f9-4d47-9f52-cea90a5e8dc4


In [ ]:
final_df = final_df_cleaned

In [ ]:
final_df.shape

(10547762, 22)

In [ ]:
final_df["CURRENT_BALANCE"].isnull().sum()

3920

In [ ]:
# List of columns to replace
columns_to_process = ['CURRENT_BALANCE', 'INCOME', 'OVERDUE_AMOUNT']

# Replace null by 0 
final_df[columns_to_process] = final_df[columns_to_process].fillna(0)

In [ ]:
final_df1 = final_df[(final_df['CURRENT_BALANCE'] >= 0) & (final_df['CURRENT_BALANCE'] <= 10000000) & (final_df['PRIMARY_NO_OF_ACCOUNTS'] <= 100)]

In [ ]:
final_df1.shape

(10084589, 22)

In [ ]:
df = final_df1

In [ ]:
#df.shape

In [ ]:
#df = df.drop('int64_field_0', axis = 1)

In [ ]:
import pandas as pd
pd.set_option('display.float_format', '{:.2f}'.format)
df.describe()

,PRIMARY_NO_OF_ACCOUNTS,PRIMARY_ACTIVE_ACCOUNTS,PRIMARY_OVERDUE_ACCOUNTS,PRIMARY_CURRENT_BALANCE,PRIMARY_SANCTIONED_AMOUNT,PRIMARY_DISTRIBUTED_AMOUNT,PRIMARY_INSTALLMENT_AMOUNT,NEW_ACCOUNTS_IN_LAST_SIX_MONTHS,NO_OF_INQUIRIES,CURRENT_BALANCE,OVERDUE_AMOUNT,INCOME,AGE_COHORT
count,10084589.00,10084589.00,10084589.00,10084589.00,10084589.00,10084589.00,10084589.00,10084589.00,10084589.00,10084589.00,10084589.00,10084589.00,10070564.00
mean,30.74,10.50,1.29,7568685.29,12145706.19,11962516.51,350836.25,3.51,1.63,166859.46,27.34,2302322.25,40.93
std,23.17,7.82,2.95,67439175.03,752916951.07,752923922.75,12790069.91,6.47,3.02,702598.18,4575.00,35793314.84,6.79
min,1.00,0.00,0.00,-559899904.00,0.00,0.00,-331462.00,0.00,0.00,0.00,-48300.00,0.00,20.00
25%,13.00,5.00,0.00,302351.00,763773.00,643390.00,13476.00,0.00,0.00,0.00,0.00,0.00,35.00
50%,24.00,9.00,0.00,1486607.00,2554540.00,2367025.00,44047.00,1.00,0.00,0.00,0.00,0.00,40.00
75%,43.00,14.00,1.00,4300743.00,6381482.00,6061585.00,128464.00,4.00,2.00,21850.00,0.00,0.00,45.00
max,99.00,82.00,78.00,26802096128.00,748806995968.00,748806995968.00,10844999649.00,65.00,42.00,10000000.00,5533702.00,999999000.00,60.00


In [ ]:
df1 = df[(df['PRIMARY_CURRENT_BALANCE'] >= 0) & (df['PRIMARY_CURRENT_BALANCE'] <= 10000000) & (df['PRIMARY_SANCTIONED_AMOUNT'] >= 0) & (df['PRIMARY_SANCTIONED_AMOUNT'] <= 10000000) & (df['PRIMARY_DISTRIBUTED_AMOUNT'] >= 0) & (df['PRIMARY_DISTRIBUTED_AMOUNT'] <= 10000000) & (df['PRIMARY_INSTALLMENT_AMOUNT'] >= 0) & (df['PRIMARY_INSTALLMENT_AMOUNT'] <= 10000000) & (df['INCOME'] >= 0) & (df['INCOME'] <= 10000000)]

In [ ]:
pd.set_option('display.float_format', '{:.2f}'.format)
df1.describe()

,PRIMARY_NO_OF_ACCOUNTS,PRIMARY_ACTIVE_ACCOUNTS,PRIMARY_OVERDUE_ACCOUNTS,PRIMARY_CURRENT_BALANCE,PRIMARY_SANCTIONED_AMOUNT,PRIMARY_DISTRIBUTED_AMOUNT,PRIMARY_INSTALLMENT_AMOUNT,NEW_ACCOUNTS_IN_LAST_SIX_MONTHS,NO_OF_INQUIRIES,CURRENT_BALANCE,OVERDUE_AMOUNT,INCOME,AGE_COHORT
count,8412439.00,8412439.00,8412439.00,8412439.00,8412439.00,8412439.00,8412439.00,8412439.00,8412439.00,8412439.00,8412439.00,8412439.00,8405166.00
mean,29.71,9.64,1.36,1784076.41,2698971.87,2533410.88,132717.52,3.72,1.63,92397.99,19.50,13720.80,40.46
std,23.31,7.13,3.07,1967066.32,2540101.81,2459734.52,422719.13,6.89,3.10,412419.70,3635.11,153390.89,6.68
min,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-11070.00,0.00,20.00
25%,12.00,4.00,0.00,217136.00,597936.00,502184.00,11018.00,0.00,0.00,0.00,0.00,0.00,35.00
50%,22.00,8.00,0.00,1044624.00,1876221.00,1706338.00,34884.00,1.00,0.00,0.00,0.00,0.00,40.00
75%,41.00,13.00,1.00,2753324.00,4186708.00,3957500.00,88216.00,4.00,2.00,14107.00,0.00,0.00,45.00
max,99.00,56.00,78.00,10000000.00,10000000.00,10000000.00,9992561.00,65.00,42.00,10000000.00,2740526.00,10000000.00,60.00


In [ ]:
df1.columns

Index(['CREDIT_REPORT_ID', 'INCOME_BAND', 'PRIMARY_NO_OF_ACCOUNTS',
       'PRIMARY_ACTIVE_ACCOUNTS', 'PRIMARY_OVERDUE_ACCOUNTS',
       'PRIMARY_CURRENT_BALANCE', 'PRIMARY_SANCTIONED_AMOUNT',
       'PRIMARY_DISTRIBUTED_AMOUNT', 'PRIMARY_INSTALLMENT_AMOUNT',
       'NEW_ACCOUNTS_IN_LAST_SIX_MONTHS', 'AVERAGE_ACCOUNT_AGE',
       'NO_OF_INQUIRIES', 'ACCOUNT_TYPE', 'CONTRIBUTOR_TYPE', 'OWNERSHIP_IND',
       'ACCOUNT_STATUS', 'CURRENT_BALANCE', 'OVERDUE_AMOUNT', 'INCOME',
       'AGE_COHORT', 'STATE', 'ROOPYA_ACCOUNT_STATUS'],
      dtype='object')

In [ ]:
df = df1.copy()

In [ ]:
# Specify the columns to consider for duplicates
columns_to_check = ['CREDIT_REPORT_ID','PRIMARY_NO_OF_ACCOUNTS', 'PRIMARY_ACTIVE_ACCOUNTS', 'PRIMARY_OVERDUE_ACCOUNTS', 'PRIMARY_CURRENT_BALANCE', 'PRIMARY_SANCTIONED_AMOUNT', 'PRIMARY_DISTRIBUTED_AMOUNT', 'PRIMARY_INSTALLMENT_AMOUNT', 'NEW_ACCOUNTS_IN_LAST_SIX_MONTHS', 'AVERAGE_ACCOUNT_AGE', 'NO_OF_INQUIRIES', 'INCOME', 'AGE_COHORT', 'STATE']

# Drop duplicates based on the specified columns
cl_df = df.drop_duplicates(subset='CREDIT_REPORT_ID')

In [ ]:
df.columns[df.isnull().any()]

Index(['INCOME_BAND', 'AGE_COHORT'], dtype='object')

In [ ]:
# Define loan categories
loan_categories = {
    'Credit Card': 'Credit Card', 'Loan on Credit Card': 'Credit Card', 'Secured Credit Card': 'Credit Card',
    'Corporate Credit Card': 'Credit Card', 'Kisan Credit Card': 'Credit Card', 'Overdraft': 'Personal Loan',
    'Personal Loan': 'Personal Loan', 'Consumer Loan': 'Personal Loan', 
    'Loan Aganist Bank Deposits': 'Personal Loan', 'OD on Savings Account': 'Personal Loan', 
    'Microfinance Personal Loan': 'Personal Loan', 'Loan to Professional': 'Personal Loan',
    'Auto Loan (Personal)': 'Auto Loan', 'Two Wheeler Loan': 'Auto Loan', 'Used Car Loan': 'Auto Loan',
    'Commercial Vehicle Loan': 'Auto Loan', 'Used Tractor Loan': 'Auto Loan', 
    'Housing Loan': 'Home Loan', 'Property Loan': 'Home Loan', 'Leasing': 'Home Loan',
    'Microfinance Housing Loan': 'Home Loan'
}

# Map the loan categories to the 'ACCOUNT_TYPE' column
df['Loan Category'] = df['ACCOUNT_TYPE'].map(loan_categories)

# Pivot the DataFrame to create separate columns for each loan category
acc_df = df.pivot_table(index='CREDIT_REPORT_ID', columns='Loan Category', aggfunc= 'size', fill_value=0)

# Reset the index to make 'CREDIT_REPORT_ID' a column again
acc_df.reset_index(inplace=True)

# Rename columns for better readability
acc_df.rename(columns={'Credit Card': 'ACCOUNT_TYPE_Credit Card', 'Home loan': 'ACCOUNT_TYPE_Home Loan', 'Auto Loan': 'ACCOUNT_TYPE_Auto Loan', 'Personal Loan': 'ACCOUNT_TYPE_Personal Loan'}, inplace=True)

In [ ]:
merged_df = acc_df.merge(df, on="CREDIT_REPORT_ID", how="left")

In [ ]:
merged_df.shape

(8326868, 27)

In [ ]:
merged_df["CREDIT_REPORT_ID"].duplicated().sum()

7222626

In [ ]:
acc_df.shape

(1104242, 5)

In [ ]:
merged_df.columns[merged_df.isnull().any()]

Index(['INCOME_BAND', 'AGE_COHORT', 'Loan Category'], dtype='object')

In [ ]:
acty_df = merged_df.drop_duplicates(subset='CREDIT_REPORT_ID')

In [ ]:
acty_df.shape

(1104242, 27)

In [ ]:
# Pivot CONTRIBUTOR_TYPE data
ct_df = df.pivot_table(index = 'CREDIT_REPORT_ID', columns = 'CONTRIBUTOR_TYPE', aggfunc ='size', fill_value = 0)

# Reset index to have 'CREDIT_REPORT_ID' as a regular column
ct_df.reset_index(inplace=True)

# Rename columns for better readability
ct_df.rename(columns={'PRB': 'CONTRIBUTOR_TYPE_PRB', 'NBF': 'CONTRIBUTOR_TYPE_NBF'}, inplace=True)

In [ ]:
merged_df = acty_df.merge(ct_df, on="CREDIT_REPORT_ID", how="left")

In [ ]:
# Pivot OWNERSHIP_IND data
oi_df = df.pivot_table(index = 'CREDIT_REPORT_ID', columns = 'OWNERSHIP_IND', aggfunc = 'size', fill_value = 0)

# Reset index to have 'CREDIT_REPORT_ID' as a regular column
oi_df.reset_index(inplace=True)

# Rename columns for better readability
oi_df.rename(columns={'Individual': 'OWNERSHIP_IND_Individual', 'Supl Card Holder': 'OWNERSHIP_IND_Supl Card Holder', 'Joint': 'OWNERSHIP_IND_Joint', 'Guarantor': 'OWNERSHIP_IND_Guarantor'}, inplace=True)

In [ ]:
merged1_df = merged_df.merge(oi_df, on="CREDIT_REPORT_ID", how="left")

In [ ]:
# Pivot ACCOUNT_STATUS data
as_df = df.pivot_table(index='CREDIT_REPORT_ID', columns='ACCOUNT_STATUS', aggfunc='size', fill_value=0)

# Reset index and remove the name of the columns index
as_df.reset_index(inplace=True)

# Rename columns for better readability
as_df.rename(columns={'Active': 'ACCOUNT_STATUS_Active', 'Closed': 'ACCOUNT_STATUS_Closed'}, inplace=True)

In [ ]:
merged2_df = merged1_df.merge(as_df, on="CREDIT_REPORT_ID", how="left")

In [ ]:
merged2_df.head(5)

,CREDIT_REPORT_ID,ACCOUNT_TYPE_Auto Loan,ACCOUNT_TYPE_Credit Card,Home Loan,ACCOUNT_TYPE_Personal Loan,INCOME_BAND,PRIMARY_NO_OF_ACCOUNTS,PRIMARY_ACTIVE_ACCOUNTS,PRIMARY_OVERDUE_ACCOUNTS,PRIMARY_CURRENT_BALANCE,...,Loan Category,CONTRIBUTOR_TYPE_NBF,CONTRIBUTOR_TYPE_PRB,OWNERSHIP_IND_Guarantor,OWNERSHIP_IND_Individual,OWNERSHIP_IND_Joint,Primary,OWNERSHIP_IND_Supl Card Holder,ACCOUNT_STATUS_Active,ACCOUNT_STATUS_Closed
0,GEOA221114CR000011523894971,0,1,0,0,H,4.00,2.00,0.00,12737.00,...,Credit Card,0,1,0,1,0,0,0,1,0
1,GEOA221114CR000111523894971,0,1,0,0,D,1.00,1.00,0.00,235.00,...,Credit Card,0,1,0,1,0,0,0,1,0
2,GEOA221114CR000211523894971,0,1,0,0,H,9.00,5.00,0.00,3052319.00,...,Credit Card,0,1,0,1,0,0,0,1,0
3,GEOA221114CR00021523894971,0,5,0,14,H,46.00,14.00,0.00,835826.00,...,NaN,15,6,0,21,0,0,0,9,12
4,GEOA221114CR00041523894971,0,0,0,3,F,15.00,5.00,4.00,399263.00,...,Personal Loan,5,0,0,5,0,0,0,1,4


In [ ]:
agg_data = df[['CREDIT_REPORT_ID', 'ACCOUNT_STATUS', 'CURRENT_BALANCE', 'OVERDUE_AMOUNT']]

def aggregation(df):
    # Create a pivot table to aggregate balances and overdue amounts
    pivot_df = df.pivot_table(index='CREDIT_REPORT_ID', columns='ACCOUNT_STATUS', 
                              values=['CURRENT_BALANCE', 'OVERDUE_AMOUNT'], 
                              aggfunc={'CURRENT_BALANCE': 'sum', 'OVERDUE_AMOUNT': 'sum'}, 
                              fill_value=0)
    
    # Flatten the multi-level column index
    pivot_df.columns = [f'{agg}_{status}' for status, agg in pivot_df.columns]
    
    # Reset the index to make 'CREDIT_REPORT_ID' a regular column
    pivot_df.reset_index(inplace=True)
    
    return pivot_df

# Call the function to aggregate balances
aggregated_data = aggregation(agg_data)

In [ ]:
merged3_df = merged2_df.merge(aggregated_data, on="CREDIT_REPORT_ID", how="left")

In [ ]:
merged2_df = merged2_df.drop(['CURRENT_BALANCE', 'OVERDUE_AMOUNT'], axis = 1)

In [ ]:
merged3_df["ROOPYA_ACCOUNT_STATUS"].value_counts()

Good    1084640
Bad       19602
Name: ROOPYA_ACCOUNT_STATUS, dtype: int64

In [ ]:
cri_df = merged3_df[['CREDIT_REPORT_ID', 'ROOPYA_ACCOUNT_STATUS']]

# Custom function to determine the final status
def categorize_customer(group):
    unique_statuses = group['ROOPYA_ACCOUNT_STATUS'].unique()

    if len(unique_statuses) == 1:
        return unique_statuses[0]
    elif 'Bad' in unique_statuses:
        return 'Bad'
    else:
        return 'Good'

result = cri_df.groupby('CREDIT_REPORT_ID').apply(categorize_customer)

# Create a new DataFrame with the summarized results
final_status_df = result.reset_index(name='ROOPYA_CUSTOMER_STATUS')

In [ ]:
merged3_df.columns

Index(['CREDIT_REPORT_ID', 'ACCOUNT_TYPE_Auto Loan',
       'ACCOUNT_TYPE_Credit Card', 'Home Loan', 'ACCOUNT_TYPE_Personal Loan',
       'INCOME_BAND', 'PRIMARY_NO_OF_ACCOUNTS', 'PRIMARY_ACTIVE_ACCOUNTS',
       'PRIMARY_OVERDUE_ACCOUNTS', 'PRIMARY_CURRENT_BALANCE',
       'PRIMARY_SANCTIONED_AMOUNT', 'PRIMARY_DISTRIBUTED_AMOUNT',
       'PRIMARY_INSTALLMENT_AMOUNT', 'NEW_ACCOUNTS_IN_LAST_SIX_MONTHS',
       'AVERAGE_ACCOUNT_AGE', 'NO_OF_INQUIRIES', 'ACCOUNT_TYPE',
       'CONTRIBUTOR_TYPE', 'OWNERSHIP_IND', 'ACCOUNT_STATUS',
       'CURRENT_BALANCE', 'OVERDUE_AMOUNT', 'INCOME', 'AGE_COHORT', 'STATE',
       'ROOPYA_ACCOUNT_STATUS', 'Loan Category', 'CONTRIBUTOR_TYPE_NBF',
       'CONTRIBUTOR_TYPE_PRB', 'OWNERSHIP_IND_Guarantor',
       'OWNERSHIP_IND_Individual', 'OWNERSHIP_IND_Joint', 'Primary',
       'OWNERSHIP_IND_Supl Card Holder', 'ACCOUNT_STATUS_Active',
       'ACCOUNT_STATUS_Closed', 'Active_CURRENT_BALANCE',
       'Closed_CURRENT_BALANCE', 'Active_OVERDUE_AMOUNT',
       '

In [ ]:
df_1 = merged3_df.drop(columns = ['INCOME_BAND', 'ACCOUNT_TYPE', 'CONTRIBUTOR_TYPE', 'OWNERSHIP_IND', 'ACCOUNT_STATUS','ROOPYA_ACCOUNT_STATUS', 'Loan Category', 'Primary'])
df_2 = final_status_df.drop(columns = ['CREDIT_REPORT_ID'])

In [ ]:
df1 = df_1.reset_index(drop=True)
df2 = df_2.reset_index(drop=True)

In [ ]:
import pandas as pd
result_df = pd.concat([df1, df2], axis = 1)

In [ ]:
final_df = result_df.rename(columns={'Home Loan': 'ACCOUNT_TYPE_Home Loan'})

In [ ]:
final_df.isnull().sum()

CREDIT_REPORT_ID                      0
ACCOUNT_TYPE_Auto Loan                0
ACCOUNT_TYPE_Credit Card              0
ACCOUNT_TYPE_Home Loan                0
ACCOUNT_TYPE_Personal Loan            0
PRIMARY_NO_OF_ACCOUNTS                0
PRIMARY_ACTIVE_ACCOUNTS               0
PRIMARY_OVERDUE_ACCOUNTS              0
PRIMARY_CURRENT_BALANCE               0
PRIMARY_SANCTIONED_AMOUNT             0
PRIMARY_DISTRIBUTED_AMOUNT            0
PRIMARY_INSTALLMENT_AMOUNT            0
NEW_ACCOUNTS_IN_LAST_SIX_MONTHS       0
AVERAGE_ACCOUNT_AGE                   0
NO_OF_INQUIRIES                       0
CURRENT_BALANCE                       0
OVERDUE_AMOUNT                        0
INCOME                                0
AGE_COHORT                         2034
STATE                                 0
CONTRIBUTOR_TYPE_NBF                  0
CONTRIBUTOR_TYPE_PRB                  0
OWNERSHIP_IND_Guarantor               0
OWNERSHIP_IND_Individual              0
OWNERSHIP_IND_Joint                   0


In [ ]:
final_df['ROOPYA_CUSTOMER_STATUS'].value_counts()

Good    1084640
Bad       19602
Name: ROOPYA_CUSTOMER_STATUS, dtype: int64

In [ ]:
print("No of customers with Auto Loan: ", len(final_df[final_df["ACCOUNT_TYPE_Auto Loan"] != 0]))
print("No of customers with Credit Card: ", len(final_df[final_df["ACCOUNT_TYPE_Credit Card"] != 0]))
print("No of customers with Home Loan: ", len(final_df[final_df["ACCOUNT_TYPE_Home Loan"] != 0]))
print("No of customers with Personal Loan: ", len(final_df[final_df["ACCOUNT_TYPE_Personal Loan"] != 0]))

No of customers with Auto Loan:  160915
No of customers with Credit Card:  831555
No of customers with Home Loan:  122333
No of customers with Personal Loan:  625293


In [ ]:
pd.set_option('display.float_format', '{:.2f}'.format)
final_df.describe()

,ACCOUNT_TYPE_Auto Loan,ACCOUNT_TYPE_Credit Card,ACCOUNT_TYPE_Home Loan,ACCOUNT_TYPE_Personal Loan,PRIMARY_NO_OF_ACCOUNTS,PRIMARY_ACTIVE_ACCOUNTS,PRIMARY_OVERDUE_ACCOUNTS,PRIMARY_CURRENT_BALANCE,PRIMARY_SANCTIONED_AMOUNT,PRIMARY_DISTRIBUTED_AMOUNT,...,OWNERSHIP_IND_Guarantor,OWNERSHIP_IND_Individual,OWNERSHIP_IND_Joint,OWNERSHIP_IND_Supl Card Holder,ACCOUNT_STATUS_Active,ACCOUNT_STATUS_Closed,Active_CURRENT_BALANCE,Closed_CURRENT_BALANCE,Active_OVERDUE_AMOUNT,Closed_OVERDUE_AMOUNT
count,1104242.00,1104242.00,1104242.00,1104242.00,1104242.00,1104242.00,1104242.00,1104242.00,1104242.00,1104242.00,...,1104242.00,1104242.00,1104242.00,1104242.00,1104242.00,1104242.00,1104242.00,1104242.00,1104242.00,1104242.00
mean,0.31,2.04,0.18,4.25,11.27,4.16,0.39,1225470.92,2100587.28,1932027.73,...,0.05,7.19,0.27,0.02,3.94,3.60,693476.74,968.39,7.35,128.34
std,1.57,4.14,0.84,27.44,10.19,3.66,1.24,1767765.01,2412553.14,2343121.10,...,0.62,31.90,1.34,0.17,11.75,22.97,2534252.76,67589.60,325.88,20394.24
min,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-44280.00,-18.00
25%,0.00,1.00,0.00,0.00,5.00,2.00,0.00,17144.25,205991.50,129833.25,...,0.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00
50%,0.00,1.00,0.00,1.00,9.00,3.00,0.00,332962.00,1075000.00,849025.50,...,0.00,3.00,0.00,0.00,2.00,1.00,27891.00,0.00,0.00,0.00
75%,0.00,2.00,0.00,3.00,15.00,6.00,0.00,1838420.75,3300564.25,3081971.25,...,0.00,5.00,0.00,0.00,4.00,2.00,318380.75,0.00,0.00,0.00
max,180.00,484.00,235.00,3375.00,99.00,56.00,78.00,10000000.00,10000000.00,10000000.00,...,186.00,3750.00,141.00,29.00,1364.00,2628.00,211213912.00,29681964.00,104558.00,17509303.00


In [ ]:
df10 = final_df[(final_df['Active_OVERDUE_AMOUNT'] >= 0)]

In [ ]:
df10 = df10.drop(columns = ['Closed_CURRENT_BALANCE', 'Closed_OVERDUE_AMOUNT'], axis = 1)

In [ ]:
df10.columns

Index(['CREDIT_REPORT_ID', 'ACCOUNT_TYPE_Auto Loan',
       'ACCOUNT_TYPE_Credit Card', 'ACCOUNT_TYPE_Home Loan',
       'ACCOUNT_TYPE_Personal Loan', 'PRIMARY_NO_OF_ACCOUNTS',
       'PRIMARY_ACTIVE_ACCOUNTS', 'PRIMARY_OVERDUE_ACCOUNTS',
       'PRIMARY_CURRENT_BALANCE', 'PRIMARY_SANCTIONED_AMOUNT',
       'PRIMARY_DISTRIBUTED_AMOUNT', 'PRIMARY_INSTALLMENT_AMOUNT',
       'NEW_ACCOUNTS_IN_LAST_SIX_MONTHS', 'AVERAGE_ACCOUNT_AGE',
       'NO_OF_INQUIRIES', 'CURRENT_BALANCE', 'OVERDUE_AMOUNT', 'INCOME',
       'AGE_COHORT', 'STATE', 'CONTRIBUTOR_TYPE_NBF', 'CONTRIBUTOR_TYPE_PRB',
       'OWNERSHIP_IND_Guarantor', 'OWNERSHIP_IND_Individual',
       'OWNERSHIP_IND_Joint', 'OWNERSHIP_IND_Supl Card Holder',
       'ACCOUNT_STATUS_Active', 'ACCOUNT_STATUS_Closed',
       'Active_CURRENT_BALANCE', 'Active_OVERDUE_AMOUNT',
       'ROOPYA_CUSTOMER_STATUS'],
      dtype='object')

In [ ]:
file_path_w = 'gs://roopya_analytics_workarea/Consolidated_Customer.csv'
df10.to_csv(file_path_w)
print("DONE")

DONE
